In [1]:
# load in packages

import pytorch_lightning as pl
import torch
import numpy as np

import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

from transformers import AutoTokenizer
from datasets import load_dataset
from torch.utils.data import Dataset

from models.transformer_clf import Transformer_CLF
from data.meta_dataset import MetaDataset

import os


In [2]:
config = {'include':['go_emotions'],
         'encoder_name':'bert-base-uncased',
         'nu':-1,
         'hidden_dims':[256, 128],
         'act_fn':'Tanh',
         'lr':1e-5,
         'batch_size':8,
         'max_epochs':1,
         'version':'go_emotions_test',
         'checkpoint_path':'./checkpoints/baselines',
         'gpu': True}

device = 'cuda' if (torch.cuda.is_available() and config['gpu']) else 'cpu'

In [3]:
class CustomDataset(Dataset):
    def __init__(self, dataset, tokenizer, device, cutoff=100):
        self.samples = []
        
        for label in sorted(dataset.keys()):
            for i, point in enumerate(dataset[label]):
                
#                 self.samples.append((point['text'], label))
#                 tokenized_input = tokenizer(point['text'],
#                                     return_tensors='pt',
#                                     padding='max_length').to(device)
#                 print(tokenized_input['input_ids'].shape)
#                 self.samples.append((tokenized_input['input_ids'].squeeze(), tokenized_input['attention_mask'].squeeze(),
#                           label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [27]:
def collate_fn(data):
    print(data)
    return tokenizer(data['text'],
                    return_tensors='pt',
                    padding='max_length').to(device)

def extract_dataloaders(dataset, tokenizer, device, batch_size=8, extract='go_emotions', shuffle=True, num_workers=0):
    data_splits = {}
    for split in dataset.datasets[extract].keys():
        data_split = CustomDataset(dataset.datasets[extract][split], tokenizer, device)
        data_split_loader = torch.utils.data.DataLoader(data_split, shuffle=shuffle, batch_size=batch_size, num_workers=num_workers, collate_fn=collate_fn)
        data_splits[split] = data_split_loader
        
    return data_splits

In [10]:
device = 'cuda' if (torch.cuda.is_available() and config['gpu']) else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(config['encoder_name'])

print("Extracting datasets")
# ToDo: process data and make sure it uses same amount of training data as protomaml
dataset = MetaDataset(include=config['include'])

print(dataset.unified.__dict__)
tokenizer_kwargs = {'return_tensors':'pt',
                                'padding':'max_length',
                                'truncation':True}

dataset.prep(tokenizer)

Extracting datasets
{'file_path': './data/datasets/unified-dataset.jsonl', 'include': ['go_emotions'], 'split_ratio': 0.8, 'verbose': False, 'first_label_only': False, 'info': []}


No config specified, defaulting to: go_emotions/simplified
Reusing dataset go_emotions (/home/bart/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e)
Loading cached processed dataset at /home/bart/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-52d24b382185989d.arrow
Loading cached processed dataset at /home/bart/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-ed2d20be2890b5ac.arrow
Loading cached processed dataset at /home/bart/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-7610501b2ccd2f6e.arrow


In [28]:
dataloaders = extract_dataloaders(dataset, tokenizer, device, config['batch_size'], config['include'][0])

In [29]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f840d27a290>,
 'validation': <torch.utils.data.dataloader.DataLoader at 0x7f83f7226350>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f83f72268d0>}

In [30]:
next(iter(dataloaders['train']))

[('Ugh I live in WA state and now I m scared I m going to get measles even though I m vaccinated , I hate anti - vaxxers', 3), ('This is why you play runescape as a kid , it teaches how to trust no one that offers free rune plate trimming', 22), ('I m sorry that happened to you but it s I can imagine I know if it was me I would forever feel Guilt .', 24), ('<NAME> voice actor by the end of the game was phenomenal . So much rage and sadness', 25), ('My opponent froze the timer , my match was only 1 second ! Also ty .', 15), ('I actually really like this , and unironically find it funny . Forgive me .', 18), ('Call me a <NAME> but I love saying that word . " No . "', 18), ('Twenty some years ago , while I was pregnant with my son , TLC aired shows like A Baby s Life . Oh , the old days ...', 17)]


TypeError: list indices must be integers or slices, not str

In [6]:
class CLFTrainer(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        # Exports the hyperparameters to a YAML file, and create "self.hparams" namespace
        self.save_hyperparameters()
        
        self.config = config
        # Create model        
        self.model = Transformer_CLF(config)
        # # Create loss module
        self.loss_module = nn.CrossEntropyLoss()


    def forward(self, text, attn_mask):
        return self.model(text, attn_mask)

        
    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), self.config["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.99)
        return [optimizer], [scheduler]


    def encode(self, text, attn_mask=None):
        return self.model.encode(text, attn_mask)
    
    def training_step(self, batch, batch_idx):
        # "batch" is the output of the train data loader.
        text, attn_mask, labels = batch
        preds = self.model(text, attn_mask)
        loss = self.loss_module(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()
        
        self.log('train_acc', acc, on_step=False, on_epoch=True) # Logs the accuracy per epoch to tensorboard (weighted average over batches)
        self.log('train_loss', loss)
        return loss # Return tensor to call ".backward" on


    def validation_step(self, batch, batch_idx):
        text, attn_mask, labels = batch
        
        preds = self.model(text, attn_mask).argmax(dim=-1)
        acc = (labels == preds).float().mean()

        self.log('val_acc', acc) # By default logs it per epoch (weighted average over batches)


    def test_step(self, batch, batch_idx):
        text, attn_mask, labels = batch
        preds = self.model(text, attn_mask).argmax(dim=-1)
        acc = (labels == preds).float().mean()
        self.log('test_acc', acc) # By default logs it per epoch (weighted average over batches), and returns it afterwards


In [7]:
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import torch.nn as nn

def train_model(config):
    """
    Function for training and testing a NLI model.
    Inputs:
        config - Namespace object from the argument parser
    """
    
    device = 'cuda' if (torch.cuda.is_available() and config['gpu']) else 'cpu'

    tokenizer = AutoTokenizer.from_pretrained(config['encoder_name'])
    
    print("Extracting datasets")
    # ToDo: process data and make sure it uses same amount of training data as protomaml
    dataset = MetaDataset(include=config['include'])
    
    print(dataset.unified.__dict__)
    tokenizer_kwargs = {'return_tensors':'pt',
                                    'padding':'max_length',
                                    'truncation':True}
    
    dataset.prep(tokenizer)
    
    print("creating dataloaders")
    data_loaders = extract_dataloaders(dataset, tokenizer, device, config['batch_size'], config['include'][0])
    train_loader = data_loaders['train']
    validation_loader = data_loaders['validation']
    test_loader = data_loaders['test']
    
    # ToDo: add n_classes
    config["n_classes"] = 27

    print('creating trainer')
    checkpoint_callback = ModelCheckpoint(dirpath=config['checkpoint_path'], save_weights_only=True, mode="max", monitor="val_acc")
    trainer = pl.Trainer(default_root_dir=os.path.join(config['checkpoint_path'], config['version']),                                
                         checkpoint_callback=checkpoint_callback, 
                         gpus=1 if str(device)=="cuda" else 0,                                                     
                         max_epochs=config['max_epochs'],                                                                           
                         progress_bar_refresh_rate=1
                         )                                                                  
    trainer.logger._log_graph = False      
    trainer.logger._default_hp_metric = None

    pl.seed_everything(1234) 
    
    model = CLFTrainer(config)
    trainer.fit(model, train_loader, validation_loader)
    
    model = CLFTrainer.load_from_checkpoint(checkpoint_callback.best_model_path)
    test_result = trainer.test(model, test_dataloaders=test_loader, verbose=False)
    
    return model

train_model(config)

Extracting datasets
{'file_path': './data/datasets/unified-dataset.jsonl', 'include': ['go_emotions'], 'split_ratio': 0.8, 'verbose': False, 'first_label_only': False, 'info': []}


No config specified, defaulting to: go_emotions/simplified
Reusing dataset go_emotions (/home/bart/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e)
Loading cached processed dataset at /home/bart/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-52d24b382185989d.arrow
Loading cached processed dataset at /home/bart/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-ed2d20be2890b5ac.arrow
Loading cached processed dataset at /home/bart/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-7610501b2ccd2f6e.arrow


creating dataloaders
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1

...

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [ ]:
device = 'cuda' if (torch.cuda.is_available() and config['gpu']) else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(config['encoder_name'])

print("Extracting datasets")
# ToDo: process data and make sure it uses same amount of training data as protomaml
dataset = MetaDataset(include=config['include'])

print(dataset.unified.__dict__)
tokenizer_kwargs = {'return_tensors':'pt',
                                'padding':'max_length',
                                'truncation':True}

dataset.prep(tokenizer)

In [ ]:
dataset.__dict__

In [ ]:
extract_dataloaders(dataset, tokenizer, device, config['batch_size'], config['include'][0])

In [ ]:
config['include']